In [3]:
import pickle as pk
# from google.colab import drive
import os

import tensorflow as tf
import numpy as np
from time import time

# drive.mount('/content/gdrive', force_remount=True)


In [4]:
!git clone https://github.com/RafaelDrumond/PeekDB.git

Cloning into 'PeekDB'...
remote: Enumerating objects: 86, done.
remote: Total 86 (delta 0), reused 0 (delta 0), pack-reused 86
Unpacking objects: 100% (86/86), done.


In [6]:
!mkdir -p val
!cp ./PeekDB/Motion/Actor2* ./val/
!ls ./val/

In [12]:
!mkdir -p tr
!cp ./PeekDB/Motion/Actor* ./tr/
!rm ./tr/Actor2*
!rm ./tr/Actor0*
!ls ./tr/

Actor10Crouching.txt  Actor4Crouching.txt  Actor7Crouching.txt
Actor10Idle.txt       Actor4Idle.txt	   Actor7Idle.txt
Actor10Running.txt    Actor4Running.txt    Actor7Running.txt
Actor10Swing.txt      Actor4Swing.txt	   Actor7Swing.txt
Actor10Walking.txt    Actor4Walking.txt    Actor7Walking.txt
Actor1Crouching.txt   Actor5Crouching.txt  Actor8Crouching.txt
Actor1Idle.txt	      Actor5Idle.txt	   Actor8Idle.txt
Actor1Running.txt     Actor5Running.txt    Actor8Running.txt
Actor1Swing.txt       Actor5Swing.txt	   Actor8Swing.txt
Actor1Walking.txt     Actor5Walking.txt    Actor8Walking.txt
Actor3Crouching.txt   Actor6Crouching.txt  Actor9Crouching.txt
Actor3Idle.txt	      Actor6Idle.txt	   Actor9Idle.txt
Actor3Running.txt     Actor6Running.txt    Actor9Running.txt
Actor3Swing.txt       Actor6Swing.txt	   Actor9Swing.txt
Actor3Walking.txt     Actor6Walking.txt    Actor9Walking.txt


In [13]:

def motionLoad(classes, directory, window = 60):

  if directory.endswith("/"):
    dire = directory
  else:
    dire = directory+"/"
  file_list = os.listdir(dire)
#   print(file_list)
  file_list = [i for i in file_list if any(s in i for s in classes)]
#   print(file_list)
  instances = []
  labels = []
  #READ FILES
  for f in file_list:
    label = [0.,0.,0.]
    index = 0
    #CHECK CLASS
    for c, cla in enumerate(classes):
      if cla in f:
        label[c] = 1.0
    #OPEN FILE
    with open(directory+f) as doc:
      lines = doc.readlines()
    end = 0
    #READ AND APPEND
    while index < len(lines):
      count = 0
      subin = []
      if index+window < len(lines):
        while count < window:
          count +=1
          out = lines[index].split("|m ")[1].split(" |class")[0].split(" ") # if you make changes look here
          out = [float(i) for i in out]
          subin.append(out)
          index +=1 # pay attention here!
        instances.append(np.array(subin))
        labels.append   (np.array(label))
      else:
        index = len(lines)
  return instances, labels

In [ ]:
classes = ["Crouching", "Idle", "Running", "Siwing", "Walking"]
actions = ["Crouching", "Running", "Swing"]
window  = 40
n_actions = len(actions)

xv, yv = motionLoad(actions, "./val/", window)
xt, yt = motionLoad(["Crouching", "Running", "Swing"], "./tr/", window)

In [21]:
n_tr = len(xt)
n_ts = len(xv)
print(n_tr, n_ts)

4816 532


In [25]:
x = tf.placeholder(tf.float32, [None, window, 20], name="Inputs")
y = tf.placeholder(tf.float32, [None, n_actions], name="Labels")

model = tf.keras.layers.LSTM(200, name="LSTM1")(x)
model = tf.layers.Dropout(.5, name="dropout1")(model)
model = tf.layers.Dense(27, name="dense1")(model)
model = tf.layers.Dense(n_actions, name="dense2")(model)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=model), name="loss")

# optimizer
optimizer = tf.train.AdamOptimizer().minimize(loss)

# Accuracy
prediction = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name="accuracy")


Begin Session
Start Epoch  1


InternalError: Dst tensor is not initialized.
	 [[node gradients_5/LSTM1_5/while/MatMul_7_grad/MatMul/StackPopV2 (defined at <ipython-input-25-a09af49bcf98>:13) ]]
	 [[node accuracy_5 (defined at <ipython-input-25-a09af49bcf98>:17) ]]

Caused by op 'gradients_5/LSTM1_5/while/MatMul_7_grad/MatMul/StackPopV2', defined at:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/abcsds/.local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/abcsds/.local/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/abcsds/.local/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/abcsds/.local/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 132, in sta

In [26]:
epochs = 20
batch_size = 50
batches = n_tr // batch_size
print("Begin Session")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    counter = 0
    tic = time()
    for epoch in range(epochs):
        print("Start Epoch {:>2}".format(epoch+1))
        t = time()
        for i in range(batches):
            tloss, tacc, _ = sess.run([loss, accuracy, optimizer],
                                         feed_dict={
                                             x: xt[i*batch_size:(i+1)*batch_size], 
                                             y: yt[i*batch_size:(i+1)*batch_size]})
        print("Training   Loss: {:>8.4f}, Training   Accuracy: {:>8.6f}".format(tloss, tacc))
        print("Epoch time: ", time() - t)
    vloss, vacc = sess.run([loss, accuracy], feed_dict={x: xv, y: yv})
    print("Validation Loss: {:>8.4f}, Validation Accuracy: {:>8.6f}".format(vloss, vacc))
    print("Training time:", time() - tic)

Begin Session
Start Epoch  1


InternalError: Dst tensor is not initialized.
	 [[node gradients_5/LSTM1_5/while/MatMul_4_grad/MatMul/StackPopV2 (defined at <ipython-input-25-a09af49bcf98>:13) ]]
	 [[node accuracy_5 (defined at <ipython-input-25-a09af49bcf98>:17) ]]

Caused by op 'gradients_5/LSTM1_5/while/MatMul_4_grad/MatMul/StackPopV2', defined at:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/abcsds/.local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/abcsds/.local/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/abcsds/.local/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/abcsds/.local/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 132, in sta

In [28]:
epochs = 20
print("Begin Session")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    counter = 0
    tic = time()
    for epoch in range(epochs):
        print("Start Epoch {:>2}".format(epoch+1))
        t = time()
        sess.run(optimizer,feed_dict={x: xt, y: yt})
        tloss, tacc = sess.run([loss, accuracy],feed_dict={x: xt, y: yt})
        vloss, vacc = sess.run([loss, accuracy], feed_dict={x: xv, y: yv})
        print("Training   Loss: {:>8.4f}, Training   Accuracy: {:>8.6f}".format(tloss, tacc))
        print("Validation Loss: {:>8.4f}, Validation Accuracy: {:>8.6f}".format(vloss, vacc))
        print("Epoch time: ", time() - t)
    print("Training time:", time() - tic)

Begin Session
Start Epoch  1


ResourceExhaustedError: OOM when allocating tensor with shape[4816,20] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node LSTM1_5/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3 (defined at <ipython-input-25-a09af49bcf98>:4) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'LSTM1_5/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3', defined at:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/abcsds/.local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/abcsds/.local/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ab